In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
plt.rc('font',family='malgun gothic')
plt.rc('axes',unicode_minus=False)
import seaborn as sns

# EDA
import klib

# Preprocessing&Feature Engineering
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, RobustScaler, MaxAbsScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectPercentile
from gensim.models import word2vec

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
import kerastuner as kt

# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, ARDRegression, BayesianRidge, Lars
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from vecstack import StackingTransformer
from vecstack import stacking

# Eveluation
from sklearn.metrics import mean_squared_error # squared=False시 RMSE
from sklearn.model_selection import cross_val_score

# Utility
import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
import pickle

In [ ]:
x_train = pd.read_csv(os.path.abspath("../input")+'/x_train_raw.csv', index_col=0)
x_test = pd.read_csv(os.path.abspath("../input")+'/x_test_raw.csv', index_col=0)
df_train = pd.read_csv(os.path.abspath("../input")+'/X_train.csv', encoding='cp949')
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949').set_index('custid')
df_test = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding='cp949')
test_id = df_test['custid'].unique()

In [ ]:
numeric = ['총구매액', '평균구매액', '최고구매금액', '총환불금액', '총환불횟수', '총할인금액', '평균할인금액', '총실구매액', '평균실구매액',
 '수입상품구매건수', '1월방문', '2월방문', '3월방문', '4월방문', '5월방문', '6월방문', '7월방문', '8월방문', '9월방문', '10월방문',
 '11월방문', '12월방문', '1_x', '2_x', '3_x', '4_x', '5_x', '6_x', '7_x', '8_x', '9_x', '10_x', '11_x', '12_x', '1_y', '2_y', '3_y',
 '4_y', '5_y', '6_y', '7_y', '8_y', '9_y', '10_y', '11_y', '12_y', '1_x.1', '2_x.1', '3_x.1', '4_x.1', '5_x.1', '6_x.1', '7_x.1',
 '8_x.1', '9_x.1', '10_x.1', '11_x.1', '12_x.1', '1_y.1', '2_y.1', '3_y.1', '4_y.1', '5_y.1', '6_y.1', '7_y.1', '8_y.1', '9_y.1',
 '10_y.1', '11_y.1', '12_y.1', '1_x.2', '2_x.2', '3_x.2', '4_x.2', '5_x.2', '6_x.2', '7_x.2', '8_x.2', '9_x.2', '10_x.2', '11_x.2',
 '12_x.2', '1_y.2', '2_y.2', '3_y.2', '4_y.2', '5_y.2', '6_y.2', '7_y.2', '8_y.2', '9_y.2', '10_y.2', '11_y.2', '12_y.2', '1_x.3',
 '2_x.3', '3_x.3', '4_x.3', '5_x.3', '6_x.3', '7_x.3', '8_x.3', '9_x.3', '10_x.3', '11_x.3', '12_x.3', '1월_구매상품다양성', '2월_구매상품다양성',
 '3월_구매상품다양성', '4월_구매상품다양성', '5월_구매상품다양성', '6월_구매상품다양성', '7월_구매상품다양성', '8월_구매상품다양성',
 '9월_구매상품다양성', '10월_구매상품다양성', '11월_구매상품다양성', '12월_구매상품다양성', '1월_구매상품개수', '2월_구매상품개수',
 '3월_구매상품개수', '4월_구매상품개수', '5월_구매상품개수', '6월_구매상품개수', '7월_구매상품개수', '8월_구매상품개수', '9월_구매상품개수',
 '10월_구매상품개수', '11월_구매상품개수', '12월_구매상품개수', '가을_x', '겨울_x', '봄_x', '여름_x', '가을_y', '겨울_y', '봄_y', '여름_y',
 '가을_x.1', '겨울_x.1', '봄_x.1', '여름_x.1', '가을_y.1', '겨울_y.1', '봄_y.1', '여름_y.1', '가을_x.2', '겨울_x.2', '봄_x.2', '여름_x.2',
 '가을_y.2', '겨울_y.2', '봄_y.2', '여름_y.2', '가을_x.3', '겨울_x.3', '봄_x.3', '여름_x.3', '가을_y.3', '겨울_y.3', '봄_y.3', '여름_y.3',
 '가을_구매상품다양성', '겨울_구매상품다양성', '봄_구매상품다양성', '여름_구매상품다양성', '가을_구매상품개수', '겨울_구매상품개수',
 '봄_구매상품개수', '여름_구매상품개수', '전반기_x', '후반기_x', '전반기_y', '후반기_y', '전반기_x.1', '후반기_x.1', '전반기_y.1',
 '후반기_y.1', '전반기_x.2', '후반기_x.2', '전반기_y.2', '후반기_y.2', '전반기_x.3', '후반기_x.3', '전반기_y.3', '후반기_y.3', '전반기_구매상품다양성',
 '후반기_구매상품다양성', '전반기_구매상품개수', '후반기_구매상품개수', '비성수기_x', '성수기_x', '비성수기_y', '성수기_y', '비성수기_x.1',
 '성수기_x.1', '비성수기_y.1', '성수기_y.1', '비성수기_x.2', '성수기_x.2', '비성수기_y.2', '성수기_y.2', '비성수기_x.3', '성수기_x.3',
 '비성수기_y.3', '성수기_y.3', '비성수기_구매상품다양성', '성수기_구매상품다양성', '비성수기_구매상품개수', '성수기_구매상품개수',
 '구매일수', '구매주기', '내점당구매금액', '내점당구매개수', '월말_x', '월중_x', '월초_x', '월말_y', '월중_y', '월초_y', '월말_x.1',
 '월중_x.1', '월초_x.1', '월말_y.1', '월중_y.1', '월초_y.1', '월말_x.2', '월중_x.2', '월초_x.2', '월말_y.2', '월중_y.2', '월초_y.2',
 '월말_x.3', '월중_x.3', '월초_x.3', '월말_y.3', '월중_y.3', '월초_y.3', '월말_구매상품다양성', '월중_구매상품다양성', '월초_구매상품다양성',
 '월말_구매상품개수', '월중_구매상품개수', '월초_구매상품개수', '월_x', '화_x', '수_x', '목_x', '금_x', '토_x', '일_x', '총방문횟수',
 '금_y', '목_y', '수_y', '월_y', '일_y', '토_y', '화_y', '금_x.1', '목_x.1', '수_x.1', '월_x.1', '일_x.1', '토_x.1', '화_x.1', '금_y.1',
 '목_y.1', '수_y.1', '월_y.1', '일_y.1', '토_y.1', '화_y.1', '금_x.2', '목_x.2', '수_x.2', '월_x.2', '일_x.2', '토_x.2', '화_x.2', '금_y.2',
 '목_y.2', '수_y.2', '월_y.2', '일_y.2', '토_y.2', '화_y.2','금_x.3', '목_x.3', '수_x.3', '월_x.3', '일_x.3', '토_x.3', '화_x.3', '금_y.3',
 '목_y.3', '수_y.3', '월_y.3', '일_y.3', '토_y.3', '화_y.3', '금_구매상품다양성', '목_구매상품다양성', '수_구매상품다양성', '월_구매상품다양성',
 '일_구매상품다양성', '토_구매상품다양성', '화_구매상품다양성', '금_구매상품개수', '목_구매상품개수', '수_구매상품개수', '월_구매상품개수',
 '일_구매상품개수', '토_구매상품개수', '화_구매상품개수', '주말_x', '주중_x', '주말_y', '주중_y', '주말_x.1', '주중_x.1', '주말_y.1',
 '주중_y.1', '주말_x.2', '주중_x.2', '주말_y.2', '주중_y.2', '주말_x.3', '주중_x.3', '주말_y.3', '주중_y.3', '주말_구매상품다양성',
 '주중_구매상품다양성', '주말_구매상품개수', '주중_구매상품개수', '9시방문', '10시방문', '11시방문', '12시방문', '13시방문', '14시방문',
 '15시방문', '16시방문', '17시방문', '18시방문', '19시방문', '20시방문', '21시방문', '22시방문', '9_y.3', '10_y.3', '11_y.3', '12_y.3',
 '13_x', '14_x', '15_x', '16_x', '17_x', '18_x', '19_x', '20_x', '21_x', '22_x', '9_x.4', '10_x.4', '11_x.4', '12_x.4', '13_y', '14_y',
 '15_y', '16_y', '17_y', '18_y', '19_y', '20_y', '21_y', '22_y', '9_y.4', '10_y.4', '11_y.4', '12_y.4', '13_x.1', '14_x.1', '15_x.1',
 '16_x.1', '17_x.1', '18_x.1', '19_x.1', '20_x.1', '21_x.1', '9_x.5', '10_x.5', '11_x.5', '12_x.5', '13_y.1', '14_y.1', '15_y.1', '16_y.1',
 '17_y.1', '18_y.1', '19_y.1', '20_y.1', '21_y.1', '22_x.1', '9_y.5', '10_y.5', '11_y.5', '12_y.5', '13_x.2', '14_x.2', '15_x.2',
 '16_x.2', '17_x.2', '18_x.2', '19_x.2', '20_x.2', '21_x.2', '22_y.1', '9_x.6', '10_x.6', '11_x.6', '12_x.6', '13_y.2', '14_y.2',
 '15_y.2', '16_y.2', '17_y.2', '18_y.2', '19_y.2', '20_y.2', '21_y.2', '22_x.2', '9_y.6', '10_y.6', '11_y.6', '12_y.6', '13_x.3',
 '14_x.3', '15_x.3', '16_x.3', '17_x.3', '18_x.3', '19_x.3', '20_x.3', '21_x.3', '22_y.2', '9시_구매상품다양성', '10시_구매상품다양성',
 '11시_구매상품다양성', '12시_구매상품다양성', '13시_구매상품다양성', '14시_구매상품다양성', '15시_구매상품다양성', '16시_구매상품다양성',
 '17시_구매상품다양성', '18시_구매상품다양성', '19시_구매상품다양성', '20시_구매상품다양성', '21시_구매상품다양성', '22시_구매상품다양성',
 '9시_구매상품개수', '10시_구매상품개수', '11시_구매상품개수', '12시_구매상품개수', '13시_구매상품개수', '14시_구매상품개수', '15시_구매상품개수',
 '16시_구매상품개수', '17시_구매상품개수', '18시_구매상품개수', '19시_구매상품개수', '20시_구매상품개수', '21시_구매상품개수', '22시_구매상품개수',
 '오전_x', '오후_x', '오전_y', '오후_y', '오전_x.1', '오후_x.1', '오전_y.1', '오후_y.1', '오전_x.2', '오후_x.2', '오전_y.2', '오후_y.2',
 '오전_x.3', '오후_x.3', '오전_y.3', '오후_y.3', '오전_구매상품다양성', '오후_구매상품다양성', '오전_구매상품개수', '오후_구매상품개수',
 '아침_구매건수_x', '저녁_구매건수_x', '점심_구매건수_x', '아침_구매건수_y', '저녁_구매건수_y', '점심_구매건수_y', '아침_구매건수_x.1',
 '저녁_구매건수_x.1', '점심_구매건수_x.1', '아침_구매건수_y.1', '저녁_구매건수_y.1', '점심_구매건수_y.1', '아침_구매건수_x.2', '저녁_구매건수_x.2',
 '점심_구매건수_x.2', '아침_구매건수_y.2', '저녁_구매건수_y.2', '점심_구매건수_y.2', '아침_구매건수_x.3', '저녁_구매건수_x.3', '점심_구매건수_x.3',
 '아침_구매건수_y.3', '저녁_구매건수_y.3', '점심_구매건수_y.3', '아침_구매건수_구매상품다양성', '저녁_구매건수_구매상품다양성', '점심_구매건수_구매상품다양성',
 '아침_구매건수_구매상품개수', '저녁_구매건수_구매상품개수', '점심_구매건수_구매상품개수', '평균쇼핑시간', '무역점방문', '본점방문',
 '신촌점방문', '천호점방문', '무역점_x', '본점_x', '신촌점_x', '천호점_x', '무역점_y', '본점_y', '신촌점_y', '천호점_y', '무역점_x.1',
 '본점_x.1', '신촌점_x.1', '천호점_x.1', '무역점_y.1', '본점_y.1', '신촌점_y.1', '천호점_y.1', '무역점_x.2', '본점_x.2', '신촌점_x.2',
 '천호점_x.2', '무역점_y.2', '본점_y.2', '신촌점_y.2', '천호점_y.2', '무역점', '본점', '신촌점', '천호점', '무역점_구매상품다양성',
 '본점_구매상품다양성', '신촌점_구매상품다양성', '천호점_구매상품다양성', '무역점_구매상품개수', '본점_구매상품개수', '신촌점_구매상품개수',
 '천호점_구매상품개수', '방문지점수', '방문지점개수', '구매상품다양성', '구매상품수', '평균할부개월수', '무이자할부평균가격', '할부결제건수',
 '1_y.3', '2_y.3', '3_y.3', '4_y.3', '5_y.3', '6_y.3', '7_y.3', '8_y.3', '9_x.7', '10_x.7', '11_x.7', '12_x.7', '1_x.4', '2_x.4', '3_x.4',
 '4_x.4', '5_x.4', '6_x.4', '7_x.4', '8_x.4', '9_y.7', '10_y.7', '11_y.7', '12_y.7', '1_y.4', '2_y.4', '3_y.4', '4_y.4','4_y.4', '5_y.4',
 '6_y.4', '7_y.4', '8_y.4', '9_x.8', '10_x.8', '11_x.8', '12_x.8', '1_x.5', '2_x.5', '3_x.5', '4_x.5', '5_x.5', '6_x.5', '7_x.5',
 '8_x.5', '9_y.8', '10_y.8', '11_y.8', '12_y.8', '1_y.5', '2_y.5', '3_y.5', '4_y.5', '5_y.5', '6_y.5', '7_y.5', '8_y.5', '9_x.9',
 '10_x.9', '11_x.9', '12_x.9', '1_x.6', '2_x.6', '3_x.6', '4_x.6', '5_x.6', '6_x.6', '7_x.6', '8_x.6', '9_y.9', '10_y.9', '11_y.9',
 '12_y.9', '1_y.6', '2_y.6', '3_y.6', '4_y.6', '5_y.6', '6_y.6', '7_y.6', '8_y.6', '9_x.10', '10_x.10', '11_x.10', '12_x.10', '1_x.7',
 '2_x.7', '3_x.7', '4_x.7', '5_x.7', '6_x.7', '7_x.7', '8_x.7', '9_y.10', '10_y.10', '11_y.10', '12_y.10', '1할부개월_구매상품다양성',
 '2할부개월_구매상품다양성', '3할부개월_구매상품다양성', '4할부개월_구매상품다양성', '5할부개월_구매상품다양성', '6할부개월_구매상품다양성',
 '7할부개월_구매상품다양성', '8할부개월_구매상품다양성', '9할부개월_구매상품다양성', '10할부개월_구매상품다양성', '11할부개월_구매상품다양성',
 '12할부개월_구매상품다양성', '1할부개월_구매상품개수', '2할부개월_구매상품개수', '3할부개월_구매상품개수', '4할부개월_구매상품개수',
 '5할부개월_구매상품개수', '6할부개월_구매상품개수', '7할부개월_구매상품개수', '8할부개월_구매상품개수', '9할부개월_구매상품개수',
 '10할부개월_구매상품개수', '11할부개월_구매상품개수', '12할부개월_구매상품개수', '가정용품_구매건수', '가정용품파트_구매건수', '골프/유니캐쥬얼_구매건수',
 '공산품_구매건수', '공산품파트_구매건수', '남성의류_구매건수', '남성정장스포츠_구매건수', '로얄부띠끄_구매건수', '로얄부틱_구매건수', '명품잡화_구매건수',
 '생식품_구매건수', '생식품파트_구매건수', '스포츠캐주얼_구매건수', '스포츠캐쥬얼_구매건수', '아동_구매건수', '아동,스포츠_구매건수',
 '아동문화_구매건수', '여성의류파트_구매건수', '여성정장_구매건수', '여성캐주얼_구매건수', '여성캐쥬얼_구매건수', '영라이브_구매건수',
 '영어덜트캐쥬얼_구매건수', '영캐릭터_구매건수', '영플라자_구매건수', '인터넷백화점_구매건수', '잡화_구매건수', '잡화파트_구매건수',
 '케주얼,구두,아동_구매건수', '패션잡화_구매건수', '가구_구매건수', '가전_구매건수', '기타바이어_구매건수', '니트단품_구매건수', '도자기크리스탈_구매건수',
 '디자이너부띠끄_구매건수', '문화완구_구매건수', '생활용품_구매건수', '섬유_구매건수', '수입명품_구매건수', '스포츠_구매건수', '엘레강스캐주얼_구매건수',
 '영캐주얼_구매건수', '유니캐주얼_구매건수', '유아동복_구매건수', '일반식품_구매건수', '장신구_구매건수', '점외_구매건수', '정장셔츠_구매건수',
 '조리식품_구매건수', '조리욕실_구매건수', '청과곡물_구매건수', '침구수예_구매건수', '캐릭터캐주얼_구매건수', '타운모피_구매건수',
 '트래디셔널캐주얼_구매건수', '피혁A_구매건수', '피혁B_구매건수', '행사장(남성)_구매건수', '행사장(아동스포츠)_구매건수', '행사장(여성정장)_구매건수',
 '행사장(여성캐주얼)_구매건수', '행사장(여성캐쥬)_구매건수', '행사장(잡화)_구매건수', '화장품_구매건수', '최애브랜드구매횟수', '판매일',
 '가정용품_총구매액', '가정용품파트_총구매액', '골프/유니캐쥬얼_총구매액', '공산품_총구매액', '공산품파트_총구매액', '남성의류_총구매액',
 '남성정장스포츠_총구매액', '로얄부띠끄_총구매액', '로얄부틱_총구매액', '명품잡화_총구매액', '생식품_총구매액', '생식품파트_총구매액',
 '스포츠캐주얼_총구매액', '스포츠캐쥬얼_총구매액', '아동_총구매액', '아동,스포츠_총구매액', '아동문화_총구매액', '여성의류파트_총구매액',
 '여성정장_총구매액', '여성캐주얼_총구매액', '여성캐쥬얼_총구매액', '영라이브_총구매액', '영어덜트캐쥬얼_총구매액', '영캐릭터_총구매액',
 '영플라자_총구매액', '인터넷백화점_총구매액', '잡화_총구매액', '잡화파트_총구매액', '케주얼,구두,아동_총구매액', '패션잡화_총구매액',
 '가정용품_평균구매액', '가정용품파트_평균구매액', '골프/유니캐쥬얼_평균구매액', '공산품_평균구매액', '공산품파트_평균구매액', '남성의류_평균구매액',
 '남성정장스포츠_평균구매액', '로얄부띠끄_평균구매액', '로얄부틱_평균구매액', '명품잡화_평균구매액', '생식품_평균구매액', '생식품파트_평균구매액',
 '스포츠캐주얼_평균구매액', '스포츠캐쥬얼_평균구매액', '아동_평균구매액', '아동,스포츠_평균구매액', '아동문화_평균구매액', '여성의류파트_평균구매액',
 '여성정장_평균구매액', '여성캐주얼_평균구매액', '여성캐쥬얼_평균구매액', '영라이브_평균구매액', '영어덜트캐쥬얼_평균구매액', '영캐릭터_평균구매액',
 '영플라자_평균구매액', '인터넷백화점_평균구매액', '잡화_평균구매액', '잡화파트_평균구매액', '케주얼,구두,아동_평균구매액', '패션잡화_평균구매액',
 '가정용품_할인금액', '가정용품파트_할인금액', '골프/유니캐쥬얼_할인금액', '공산품_할인금액', '공산품파트_할인금액', '남성의류_할인금액',
 '남성정장스포츠_할인금액', '로얄부띠끄_할인금액', '로얄부틱_할인금액', '명품잡화_할인금액', '생식품_할인금액', '생식품파트_할인금액',
 '스포츠캐주얼_할인금액', '스포츠캐쥬얼_할인금액', '아동_할인금액', '아동,스포츠_할인금액', '아동문화_할인금액', '여성의류파트_할인금액',
 '여성정장_할인금액', '여성캐주얼_할인금액', '여성캐쥬얼_할인금액', '영라이브_할인금액', '영어덜트캐쥬얼_할인금액', '영캐릭터_할인금액',
 '영플라자_할인금액', '인터넷백화점_할인금액', '잡화_할인금액', '잡화파트_할인금액', '케주얼,구두,아동_할인금액', '패션잡화_할인금액',
 '가정용품_구매상품다양성', '가정용품파트_구매상품다양성', '골프/유니캐쥬얼_구매상품다양성', '공산품_구매상품다양성', '공산품파트_구매상품다양성',
 '남성의류_구매상품다양성', '남성정장스포츠_구매상품다양성', '로얄부띠끄_구매상품다양성', '로얄부틱_구매상품다양성', '명품잡화_구매상품다양성',
 '생식품_구매상품다양성', '생식품파트_구매상품다양성', '스포츠캐주얼_구매상품다양성', '스포츠캐쥬얼_구매상품다양성', '아동_구매상품다양성',
 '아동,스포츠_구매상품다양성', '아동문화_구매상품다양성', '여성의류파트_구매상품다양성', '여성정장_구매상품다양성', '여성캐주얼_구매상품다양성',
 '여성캐쥬얼_구매상품다양성', '영라이브_구매상품다양성', '영어덜트캐쥬얼_구매상품다양성', '영캐릭터_구매상품다양성', '영플라자_구매상품다양성',
 '인터넷백화점_구매상품다양성', '잡화_구매상품다양성', '잡화파트_구매상품다양성', '케주얼,구두,아동_구매상품다양성', '패션잡화_구매상품다양성',
 '가정용품_구매상품개수', '가정용품파트_구매상품개수', '골프/유니캐쥬얼_구매상품개수', '공산품_구매상품개수', '공산품파트_구매상품개수',
 '남성의류_구매상품개수', '남성정장스포츠_구매상품개수', '로얄부띠끄_구매상품개수', '로얄부틱_구매상품개수', '명품잡화_구매상품개수',
 '생식품_구매상품개수', '생식품파트_구매상품개수', '스포츠캐주얼_구매상품개수', '스포츠캐쥬얼_구매상품개수', '아동_구매상품개수',
 '아동,스포츠_구매상품개수', '아동문화_구매상품개수', '여성의류파트_구매상품개수', '여성정장_구매상품개수', '여성캐주얼_구매상품개수',
 '여성캐쥬얼_구매상품개수', '영라이브_구매상품개수', '영어덜트캐쥬얼_구매상품개수', '영캐릭터_구매상품개수', '영플라자_구매상품개수',
 '인터넷백화점_구매상품개수', '잡화_구매상품개수', '잡화파트_구매상품개수', '케주얼,구두,아동_구매상품개수', '패션잡화_구매상품개수']
prop = ['수입상품구매비율', '1월방문_prop', '2월방문_prop', '3월방문_prop', '4월방문_prop', '5월방문_prop', '6월방문_prop', '7월방문_prop',
 '8월방문_prop', '9월방문_prop', '10월방문_prop', '11월방문_prop', '12월방문_prop', '성수기방문비율', '비성수기방문비율', '전반기방문비율',
 '후반기방문비율', '가을_prop', '겨울_prop', '봄_prop', '여름_prop', '월_prop', '화_prop', '수_prop', '목_prop', '금_prop', '토_prop',
 '일_prop', '주말방문비율', '9시방문_prop', '10시방문_prop', '11시방문_prop', '12시방문_prop', '13시방문_prop', '14시방문_prop', '15시방문_prop',
 '16시방문_prop', '17시방문_prop', '18시방문_prop', '19시방문_prop', '20시방문_prop', '21시방문_prop', '22시방문_prop', '아침_구매건수_prop',
 '저녁_구매건수_prop', '점심_구매건수_prop', '오전방문비율', '오후방문비율', '무역점방문_prop', '본점방문_prop', '신촌점방문_prop',
 '천호점방문_prop', '할부결제비율', '할인율_x', '할인율_y',]

In [ ]:
x_train_num = x_train[numeric+prop]
x_train_cat = x_train.drop(columns=numeric, axis=1)
x_test_num = x_test[numeric+prop]
x_test_cat = x_test.drop(columns=numeric, axis=1)

In [ ]:
print(x_train_num.shape, x_test_num.shape,'\n',x_train_cat.shape,x_test_cat.shape)

## Categorical Features

### Feature Selection

In [ ]:
features_cat = x_train_cat
# 사용할 모델 설정 (속도가 빠른 모델 사용 권장)
model = Ridge(random_state=0)

# 각 특성과 타깃(class) 사이에 유의한 통계적 관계가 있는지 계산하여 특성을 선택하는 방법 
# feature 개수 바꿔가며 성능 test한다.
cv_scores = []
for p in tqdm(range(5,100,1)):
    X_new = SelectPercentile(percentile=p).fit_transform(x_train_cat, y_train)    
    cv_score = cross_val_score(model, X_new, y_train, scoring='neg_mean_squared_error', cv=5).mean()
    cv_scores.append((p,cv_score))

# Print the best percentile
best_score = cv_scores[np.argmax([score for _, score in cv_scores])]
print(best_score)

# Plot the performance change with p
plt.plot([k for k, _ in cv_scores], [score for _, score in cv_scores])
plt.xlabel('Percent of features')
plt.grid()

In [ ]:
# 과적합을 피하기 위해 최적의 p값 주변의 값을 선택하는게 더 나은 결과를 얻을 수 있다. 
fs = SelectPercentile(percentile=best_score[0]).fit(x_train_cat, y_train)
x_train_cat = fs.transform(x_train_cat)
x_test_cat = fs.transform(x_test_cat)

print(x_train_cat.shape)
print(features_cat.columns[fs.get_support()].tolist())

In [ ]:
pd.DataFrame(x_train_cat,columns=features_cat.columns[fs.get_support()].tolist()).to_csv('x_train_cat.csv')
pd.DataFrame(x_test_cat,columns=features_cat.columns[fs.get_support()].tolist()).to_csv('x_test_cat.csv')

### Model Tuning

In [ ]:
x_train_cat = np.array(x_train_cat)
x_test_cat = np.array(x_test_cat)

In [ ]:
x_train_cat2, x_dev_cat, y_train2, y_dev = train_test_split(x_train_cat, y_train, test_size=0.3, random_state=0)

In [ ]:
pbounds = {
    'alpha':(0,50)
}
def rid_opt(alpha):
    params = {
        'alpha':alpha
    }
    rid = Ridge(random_state=0, **params)
    rid.fit(x_train_cat2,y_train2)
    score = mean_squared_error(rid.predict(x_dev_cat),y_dev,squared=False)
    return -score
BO_rid = BayesianOptimization(rid_opt, pbounds, random_state=0)
BO_rid.maximize(init_points=50, n_iter=50) # init_points: exploration, n_iter: iteration

In [ ]:
BO_rid.max

In [ ]:
pbounds = {
    'alpha':(0,50)
}
def las_opt(alpha):
    params = {
        'alpha':alpha
    }
    las = Lasso(random_state=0, **params)
    las.fit(x_train_cat2,y_train2)
    score = mean_squared_error(las.predict(x_dev_cat),y_dev,squared=False)
    return -score
BO_las = BayesianOptimization(las_opt, pbounds, random_state=0)
BO_las.maximize(init_points=50, n_iter=50) # init_points: exploration, n_iter: iteration

In [ ]:
BO_las.max

In [ ]:
pbounds = {
    'alpha':(0,50)
}
def ela_opt(alpha):
    params = {
        'alpha':alpha
    }
    ela = ElasticNet(random_state=0, **params)
    ela.fit(x_train_cat2,y_train2)
    score = mean_squared_error(ela.predict(x_dev_cat),y_dev,squared=False)
    return -score
BO_ela = BayesianOptimization(ela_opt, pbounds, random_state=0)
BO_ela.maximize(init_points=50, n_iter=50) # init_points: exploration, n_iter: iteration

In [ ]:
BO_ela.max

In [ ]:
pbounds = {
    'n_iter':(100,1000),
    'alpha_1':(0,50),
    'alpha_2':(0,50),
    'lambda_1':(0,10),
    'lambda_2':(0,10)
}
def ard_opt(n_iter,alpha_1,alpha_2,lambda_1,lambda_2):
    params = {
        'n_iter':int(round(n_iter)),
        'alpha_1':alpha_1,
        'alpha_2':alpha_2,
        'lambda_1':lambda_1,
        'lambda_2':lambda_2
    }
    ard = ARDRegression(**params)
    ard.fit(x_train_cat2,y_train2)
    score = mean_squared_error(ard.predict(x_dev_cat),y_dev,squared=False)
    return -score
BO_ard = BayesianOptimization(ard_opt, pbounds, random_state=0)
BO_ard.maximize(init_points=50, n_iter=50) # init_points: exploration, n_iter: iteration

In [ ]:
BO_ard.max

In [ ]:
pbounds = {
    'n_iter':(100,1000),
    'alpha_1':(0,50),
    'alpha_2':(0,50),
    'lambda_1':(0,10),
    'lambda_2':(0,10)
}
def bay_opt(n_iter,alpha_1,alpha_2,lambda_1,lambda_2):
    params = {
        'n_iter':int(round(n_iter)),
        'alpha_1':alpha_1,
        'alpha_2':alpha_2,
        'lambda_1':lambda_1,
        'lambda_2':lambda_2
    }
    bay = BayesianRidge(**params)
    bay.fit(x_train_cat2,y_train2)
    score = mean_squared_error(bay.predict(x_dev_cat),y_dev,squared=False)
    return -score
BO_bay = BayesianOptimization(bay_opt, pbounds, random_state=0)
BO_bay.maximize(init_points=50, n_iter=50)

In [ ]:
BO_bay.max

In [ ]:
pbounds = {
    'n_estimators':(100,1000),
    'learning_rate':(0,1),
    'max_depth':(2, 32),
    'num_leaves':(2, 64),
    'min_child_samples':(10, 200),
    'min_child_weight':(1, 50),
    'subsample':(0.5, 1),
    'colsample_bytree':(0.5, 1),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha':(0.01, 50)
}
def lgbm_opt(n_estimators, learning_rate, max_depth, num_leaves, min_child_samples, min_child_weight,
             subsample, colsample_bytree, max_bin, reg_lambda, reg_alpha):
    params = {
        "n_estimators":int(round(n_estimators)), 
        "learning_rate":learning_rate,
        'max_depth':int(round(max_depth)),
        'num_leaves':int(round(num_leaves)),
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample':max(min(subsample, 1), 0),
        'colsample_bytree':max(min(colsample_bytree, 1), 0),
        'reg_lambda': reg_lambda,
        'reg_alpha': reg_alpha
    }
    lgbm = LGBMRegressor(random_state=0, **params)
    lgbm.fit(x_train_cat2,y_train2)
    score = mean_squared_error(lgbm.predict(x_dev_cat),y_dev,squared=False)
    return -score
BO_lgbm = BayesianOptimization(lgbm_opt, pbounds, random_state=0)
BO_lgbm.maximize(init_points=50, n_iter=50)

In [ ]:
BO_lgbm.max

In [ ]:
max_params_rid = BO_rid.max['params']
max_params_las = BO_las.max['params']
max_params_ela = BO_ela.max['params']
max_params_ard = BO_ard.max['params']
max_params_bay = BO_bay.max['params']
max_params_lgbm = BO_lgbm.max['params']

In [ ]:
max_params_ard['n_iter'] = int(round(max_params_ard['n_iter']))

max_params_bay['n_iter'] = int(round(max_params_bay['n_iter']))

max_params_lgbm['num_leaves'] = int(round(max_params_lgbm['num_leaves']))
max_params_lgbm['n_estimators'] = int(round(max_params_lgbm['n_estimators']))
max_params_lgbm['max_depth'] = int(round(max_params_lgbm['max_depth']))
max_params_lgbm['min_child_samples'] = int(round(max_params_lgbm['min_child_samples']))
max_params_lgbm['min_child_weight'] = int(round(max_params_lgbm['min_child_weight']))
max_params_lgbm['max_bin'] = int(round(max_params_lgbm['max_bin']))
max_params_lgbm['subsample'] = max(min(max_params_lgbm['subsample'], 1), 0)
max_params_lgbm['colsample_bytree'] = max(min(max_params_lgbm['colsample_bytree'], 1), 0)

In [ ]:
print(max_params_rid,'\n',max_params_las,'\n',max_params_ela,'\n',max_params_ard,'\n',max_params_bay,'\n',max_params_lgbm,'\n')

In [ ]:
max_params_rid = {'alpha': 2.141450519257165} 
max_params_las = {'alpha': 0.0031679860402866744} 
max_params_ela = {'alpha': 0.0031679860402866744} 
max_params_ard = {'alpha_1': 34.76870111024178, 'alpha_2': 40.063650975282826, 'lambda_1': 0.6487969603784871, 'lambda_2': 9.930508557701192, 'n_iter': 774} 
max_params_bay = {'alpha_1': 49.03743486079745, 'alpha_2': 40.03692702558726, 'lambda_1': 0.0, 'lambda_2': 10.0, 'n_iter': 747} 
max_params_lgbm = {'colsample_bytree': 0.7842169744343243, 'learning_rate': 0.018789800436355142, 'max_bin': 313, 'max_depth': 20, 'min_child_samples': 127, 'min_child_weight': 47, 'n_estimators': 714, 'num_leaves': 24, 'reg_alpha': 21.857227370429083, 'reg_lambda': 6.976614328076722, 'subsample': 0.5301127358146349}

In [ ]:
regs_tuned = [Ridge(random_state=0, **max_params_rid),Lasso(random_state=0, **max_params_las),ElasticNet(random_state=0, **max_params_ela),
             ARDRegression(**max_params_ard),BayesianRidge(**max_params_bay),LGBMRegressor(random_state=0,**max_params_lgbm),CatBoostRegressor(random_state=0)]
regs_tuned = [(str(reg).split('(')[0], reg) for reg in regs_tuned]
regs_tuned[-1] = list(regs_tuned[-1])
regs_tuned[-1][0] = 'CatBoostRegressor'
regs_tuned[-1] = tuple(regs_tuned[-1])

regs_trained = [(name, reg.fit(x_train_cat2,y_train2), float(mean_squared_error(reg.predict(x_dev_cat),y_dev,squared=False))) 
                    for name, reg in tqdm(regs_tuned.copy())]

In [ ]:
regs_tuned = [Ridge(random_state=0, **max_params_rid),Lasso(random_state=0, **max_params_las),ElasticNet(random_state=0, **max_params_ela),
             ARDRegression(**max_params_ard),BayesianRidge(**max_params_bay),LGBMRegressor(random_state=0,**max_params_lgbm),CatBoostRegressor(random_state=0)]
regs_tuned = [(str(reg).split('(')[0], reg) for reg in regs_tuned]
regs_tuned[-1] = list(regs_tuned[-1])
regs_tuned[-1][0] = 'CatBoostRegressor'
regs_tuned[-1] = tuple(regs_tuned[-1])

regs_trained_for_submissions = [(name, reg.fit(x_train_cat,y_train)) for name, reg in tqdm(regs_tuned.copy())]

In [ ]:
regs_tuned = [Ridge(random_state=0, **max_params_rid),Lasso(random_state=0, **max_params_las),ElasticNet(random_state=0, **max_params_ela),
             ARDRegression(**max_params_ard),BayesianRidge(**max_params_bay),LGBMRegressor(random_state=0,**max_params_lgbm),CatBoostRegressor(random_state=0)]
regs_tuned = [(str(reg).split('(')[0], reg) for reg in regs_tuned]
regs_tuned[-1] = list(regs_tuned[-1])
regs_tuned[-1][0] = 'CatBoostRegressor'
regs_tuned[-1] = tuple(regs_tuned[-1])

### Model Ensemble

In [ ]:
pred_results = []
for name, reg, reg_score in regs_trained:
    pred = list(reg.predict(x_dev_cat))
    name = f'{name} \n({reg_score:.4f})'
    pred_results.append(pd.Series(pred, name=name))
ensemble_results = pd.concat(pred_results, axis=1)
ensemble_results = ensemble_results.applymap(lambda x: float(x))

# 모형의 예측값 간의 상관관계를 보기 위해 hitmap을 도식한다.
plt.figure(figsize = (8,6))
g = sns.heatmap(ensemble_results.corr(), annot=True, cmap='Blues')
g.set_title("Correlation between models")
plt.show()

In [ ]:
corr = (ensemble_results.corr().sum()-1)/(ensemble_results.corr().shape[0]-1)
names = corr.index
rmse = np.array(corr.index.str[-7:-1]).astype(float)
df = pd.DataFrame({'model': names, 'rmse': rmse, 'cor': corr})        

plt.figure(figsize=(8,6))
g = sns.scatterplot(x="cor", y="rmse", data=df, s=40, color='red')
for line in range(0, df.shape[0]):
     g.text(df.cor[line]+0.003, df.rmse[line]-0.003, 
            df.model[line], horizontalalignment='left', 
            size='medium', color='black', weight='semibold')
        
plt.xlim((df.cor.min()-0.01,df.cor.max()+0.01))
plt.ylim((df.rmse.min()-0.01,df.rmse.max()+0.01))
plt.xlabel('Mean Agreement')
plt.ylabel('RMSE')
plt.grid()
plt.show()

In [ ]:
selected = [#'LinearRegression',
            'Ridge',
            #'Lasso',
            #'ElasticNet',
            #'ARDRegression',
            #'BayesianRidge',
            #'RandomForestRegressor',
            #'XGBRegressor',
            'LGBMRegressor',
            'CatBoostRegressor'
            ]
models_for_ensemble = [(name,reg) for name,reg,score in regs_trained if name in selected]
results_for_ensemble = []
for name, model in models_for_ensemble:
    results_for_ensemble.append(model.predict(x_dev_cat))
avg_test = (results_for_ensemble[0].flatten()+results_for_ensemble[1]+results_for_ensemble[2])/len(results_for_ensemble)
score = mean_squared_error(avg_test, y_dev, squared=False)
score

In [ ]:
# 최적의 가중치 찾기 
selected = [#'LinearRegression',
            'Ridge',
            #'Lasso',
            #'ElasticNet',
            #'ARDRegression',
            #'BayesianRidge',
            #'RandomForestRegressor',
            #'XGBRegressor',
            'LGBMRegressor',
            'CatBoostRegressor'
            ]
models_for_ensemble = [(name,reg) for name,reg,score in regs_trained if name in selected]
weights_avg = []
rmse_best = 1000
for i in tqdm(range(1, 30, 1)):
    for j in range(1, 30, 1):
        for k in range(1, 30, 1):
            if (i+j+k) != 30:
                continue
            pred = (models_for_ensemble[0][1].predict(x_dev_cat).flatten() * i + models_for_ensemble[1][1].predict(x_dev_cat) * j
                    + models_for_ensemble[2][1].predict(x_dev_cat) * k)/30
            rmse = np.sqrt(mean_squared_error(y_dev, pred))
            if rmse < rmse_best:
                weights_avg = [i,j,k]
                rmse_best = rmse 
                print(rmse, i,j,k)            

print(rmse_best, weights_avg)

In [ ]:
weights_avg = [9,17,4]

In [ ]:
selected = [#'LinearRegression',
            'Ridge',
            #'Lasso',
            #'ElasticNet',
            #'ARDRegression',
            #'BayesianRidge',
            #'RandomForestRegressor',
            #'XGBRegressor',
            'LGBMRegressor',
            'CatBoostRegressor'
            ]
models_for_ensemble = [(name,reg) for name,reg,score in regs_trained if name in selected]
i,j,k = weights_avg
avg_test_result = (models_for_ensemble[0][1].predict(x_dev_cat).flatten()*i + models_for_ensemble[1][1].predict(x_dev_cat)*j + 
       models_for_ensemble[2][1].predict(x_dev_cat)*k)/30

### Deep Neural Network
'dnn_cat.krs'

In [ ]:
def reset_seeds(reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  # optional

    np.random.seed(99)
    # seed를 잘 설정하면 성능이 더 잘 오른다.
    random.seed(9)
    tf.compat.v1.set_random_seed(16)
#    os.environ['CUDA_VISIBLE_DEVICES'] = ''  # for GPU
    print("RANDOM SEEDS RESET")  # optional
   
reset_seeds()

In [ ]:
i = int(round(x_train_cat2.shape[0] * 0.8,0))
x_val_cat, y_val = x_train_cat2[i:], y_train2[i:]
x_train_cat3, y_train3 = x_train_cat2[:i], y_train2[:i]

In [ ]:
def model_fn(hp):
    inputs = tf.keras.Input(shape=(x_train_cat3.shape[1],))
    x = inputs
    for i in range(hp.Int('num_layers', 2, 4, step=1)):
        x = tf.keras.layers.Dense(hp.Int('unit_'+str(i), 16, 256, step=16),
                               activation=hp.Choice('activation',['relu','tanh']))(x)
        x = tf.keras.layers.Dropout(hp.Float('dropout_'+str(i), 0, 0.7, step=0.1, default=0.5))(x)
    outputs = tf.keras.layers.Dense(1, activation='linear')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(loss='mse', 
                  optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5, 1e-6])), 
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

# keras tuner는 튜닝 종류가 4종류가 있음: hyperband, grid search, random search, bayesian optimization
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_root_mean_squared_error', direction="min"), 
                     max_epochs=30,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
# objective: 튜닝 기준, hyperband_iterations:이거 자체에서 2번 반복
# overwrite: False시, 기존을 근거로 해 재학습 안시킴

tuner.search(x_train_cat3, y_train3, validation_data=(x_val_cat, y_val),
             callbacks=[tf.keras.callbacks.EarlyStopping()])
# 빨리 끝내려고 파라미터 저렇게 설정한 것임

In [ ]:
tuner.results_summary(1) # 1= 제일 성능이 좋은 놈

In [ ]:
# Loss & RMSE
dnn = tuner.get_best_models(1)[0] # best model 중 가장 좋은 모델
dnn.evaluate(x_dev_cat, y_dev)

In [ ]:
dnn.summary()

In [ ]:
sns.heatmap(pd.DataFrame(dict(zip(['dnn','avg'],[dnn.predict(x_dev_cat).flatten(),avg_test_result]))).corr(), annot=True, cmap='Blues')
plt.show()

In [ ]:
mean_squared_error((dnn.predict(x_dev_cat).flatten()+avg_test_result)/2,y_dev,squared=False)

In [ ]:
weights_da = []
rmse_best = 1000
for i in tqdm(range(1,20)):
    for j in range(1,20):
        if i+j!=20:
            continue
        pred = (avg_test_result*i + dnn.predict(x_dev_cat).flatten()*j)/20
        rmse = np.sqrt(mean_squared_error(y_dev, pred))
        if rmse < rmse_best:
            weights_da = [i,j]
            rmse_best = rmse 
            print(rmse, i,j)     
print(rmse_best, weights_da)

### Stacking

In [ ]:
selected = [#'LinearRegression',
            'Ridge',
            #'Lasso',
            #'ElasticNet',
            #'ARDRegression',
            #'BayesianRidge',
            #'RandomForestRegressor',
            #'XGBRegressor',
            'LGBMRegressor',
            'CatBoostRegressor'
            ]
stack_estimators = [reg for name,reg,score in regs_trained if name in selected]

In [ ]:
S_train, S_test = stacking(stack_estimators,
                           x_train_cat, y_train, x_test_cat,
                           regression=True, n_folds=5, stratified=True, shuffle=True,
                           random_state=0, verbose=0)

In [ ]:
S_train2, S_dev = stacking(stack_estimators,
                           x_train_cat2, y_train2, x_dev_cat,
                           regression=True, n_folds=5, stratified=True, shuffle=True,
                           random_state=0, verbose=0)

In [ ]:
pbounds = {
    'alpha':(0,50)
}
def rid_stk_opt(alpha):
    params = {
        'alpha':alpha
    }
    rid = Ridge(random_state=0, **params)
    skf = StratifiedKFold(n_splits=5 , shuffle=True, random_state=1)
    score = cross_val_score(rid, S_train2, y_train2, scoring='neg_mean_squared_error', cv=skf, n_jobs=-1)
    return -np.sqrt(-np.mean(score))
BO_stk_rid = BayesianOptimization(rid_stk_opt, pbounds, random_state=0)
BO_stk_rid.maximize(init_points=50, n_iter=50) # init_points: exploration, n_iter: iteration

In [ ]:
BO_stk_rid.max

In [ ]:
pbounds = {
    'alpha':(0,50)
}
def las_stk_opt(alpha):
    params = {
        'alpha':alpha
    }
    las = Lasso(random_state=0, **params)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    score = cross_val_score(las,S_train2, y_train2, scoring='neg_mean_squared_error', cv=skf, n_jobs=-1)
    return -np.sqrt(-np.mean(score))
BO_stk_las = BayesianOptimization(las_stk_opt, pbounds, random_state=0)
BO_stk_las.maximize(init_points=50, n_iter=50) # init_points: exploration, n_iter: iteration

In [ ]:
BO_stk_las.max

In [ ]:
pbounds = {
    'alpha':(0,50)
}
def ela_stk_opt(alpha):
    params = {
        'alpha':alpha
    }
    ela = ElasticNet(random_state=0, **params)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    score = cross_val_score(ela, S_train2, y_train2, scoring='neg_mean_squared_error', cv=skf, n_jobs=-1)
    return -np.sqrt(-np.mean(score))
BO_stk_ela = BayesianOptimization(ela_stk_opt, pbounds, random_state=0)
BO_stk_ela.maximize(init_points=50, n_iter=50) # init_points: exploration, n_iter: iteration

In [ ]:
BO_stk_ela.max

In [ ]:
pbounds = {
    'n_iter':(100,1000),
    'alpha_1':(0,50),
    'alpha_2':(0,50),
    'lambda_1':(0,10),
    'lambda_2':(0,10)
}
def ard_stk_opt(n_iter,alpha_1,alpha_2,lambda_1,lambda_2):
    params = {
        'n_iter':int(round(n_iter)),
        'alpha_1':alpha_1,
        'alpha_2':alpha_2,
        'lambda_1':lambda_1,
        'lambda_2':lambda_2
    }
    ard = ARDRegression(**params)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    score = cross_val_score(ard, S_train2, y_train2, scoring='neg_mean_squared_error', cv=skf, n_jobs=-1)
    return -np.sqrt(-np.mean(score))
BO_stk_ard = BayesianOptimization(ard_stk_opt, pbounds, random_state=0)
BO_stk_ard.maximize(init_points=50, n_iter=50) # init_points: exploration, n_iter: iteration

In [ ]:
BO_stk_ard.max

In [ ]:
pbounds = {
    'n_iter':(100,1000),
    'alpha_1':(0,50),
    'alpha_2':(0,50),
    'lambda_1':(0,10),
    'lambda_2':(0,10)
}
def bay_stk_opt(n_iter,alpha_1,alpha_2,lambda_1,lambda_2):
    params = {
        'n_iter':int(round(n_iter)),
        'alpha_1':alpha_1,
        'alpha_2':alpha_2,
        'lambda_1':lambda_1,
        'lambda_2':lambda_2
    }
    bay = BayesianRidge(**params)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    score = cross_val_score(bay,S_train2, y_train2, scoring='neg_mean_squared_error', cv=skf, n_jobs=-1)
    return -np.sqrt(-np.mean(score))
BO_stk_bay = BayesianOptimization(bay_stk_opt, pbounds, random_state=0)
BO_stk_bay.maximize(init_points=50, n_iter=50)

In [ ]:
BO_stk_bay.max

In [ ]:
pbounds = {
    'n_estimators':(100,1000),
    'learning_rate':(0.0000000000000000001,1),
    'max_depth':(2, 32),
    'num_leaves':(2, 64),
    'min_child_samples':(10, 200),
    'min_child_weight':(1, 50),
    'subsample':(0.5, 1),
    'colsample_bytree':(0.5, 1),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha':(0.01, 50)
}
def lgbm_stk_opt(n_estimators, learning_rate, max_depth, num_leaves, min_child_samples, min_child_weight,
             subsample, colsample_bytree, max_bin, reg_lambda, reg_alpha):
    params = {
        "n_estimators":int(round(n_estimators)), 
        "learning_rate":learning_rate,
        'max_depth':int(round(max_depth)),
        'num_leaves':int(round(num_leaves)),
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample':max(min(subsample, 1), 0),
        'colsample_bytree':max(min(colsample_bytree, 1), 0),
        'reg_lambda': reg_lambda,
        'reg_alpha': reg_alpha
    }
    lgbm = LGBMRegressor(random_state=0, **params)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    score = cross_val_score(lgbm, S_train2, y_train2, scoring='neg_mean_squared_error', cv=skf, n_jobs=-1)
    return -np.sqrt(-np.mean(score))
BO_stk_lgbm = BayesianOptimization(lgbm_stk_opt, pbounds, random_state=0)
BO_stk_lgbm.maximize(init_points=50, n_iter=50)

In [ ]:
BO_stk_lgbm.max

In [ ]:
max_params_rid = BO_stk_rid.max['params']
max_params_las = BO_stk_las.max['params']
max_params_ela = BO_stk_ela.max['params']
max_params_ard = BO_stk_ard.max['params']
max_params_bay = BO_stk_bay.max['params']
max_params_lgbm = BO_stk_lgbm.max['params']

In [ ]:
max_params_ard['n_iter'] = int(round(max_params_ard['n_iter']))

max_params_bay['n_iter'] = int(round(max_params_bay['n_iter']))

max_params_lgbm['num_leaves'] = int(round(max_params_lgbm['num_leaves']))
max_params_lgbm['n_estimators'] = int(round(max_params_lgbm['n_estimators']))
max_params_lgbm['max_depth'] = int(round(max_params_lgbm['max_depth']))
max_params_lgbm['min_child_samples'] = int(round(max_params_lgbm['min_child_samples']))
max_params_lgbm['min_child_weight'] = int(round(max_params_lgbm['min_child_weight']))
max_params_lgbm['max_bin'] = int(round(max_params_lgbm['max_bin']))
max_params_lgbm['subsample'] = max(min(max_params_lgbm['subsample'], 1), 0)
max_params_lgbm['colsample_bytree'] = max(min(max_params_lgbm['colsample_bytree'], 1), 0)

In [ ]:
print(max_params_rid,'\n',max_params_las,'\n',max_params_ela,'\n',max_params_ard,'\n',max_params_bay,'\n',max_params_lgbm)

In [ ]:
stks_tuned = [Ridge(random_state=0, **max_params_rid),Lasso(random_state=0, **max_params_las),ElasticNet(random_state=0, **max_params_ela),
             ARDRegression(**max_params_ard),BayesianRidge(**max_params_bay),LGBMRegressor(random_state=0,**max_params_lgbm),CatBoostRegressor(random_state=0)]
stks_tuned = [(str(stk).split('(')[0], stk) for stk in stks_tuned]
stks_tuned[-1] = list(stks_tuned[-1])
stks_tuned[-1][0] = 'CatBoostRegressor'
stks_tuned[-1] = tuple(stks_tuned[-1])

stks_trained = [(name, stk.fit(S_train2, y_train2),mean_squared_error(stk.predict(S_dev),y_dev,squared=False))
                    for name, stk in tqdm(stks_tuned.copy())]

In [ ]:
stks_tuned = [Ridge(random_state=0, **max_params_rid),Lasso(random_state=0, **max_params_las),ElasticNet(random_state=0, **max_params_ela),
             ARDRegression(**max_params_ard),BayesianRidge(**max_params_bay),LGBMRegressor(random_state=0,**max_params_lgbm),CatBoostRegressor(random_state=0)]
stks_tuned = [(str(stk).split('(')[0], stk) for stk in stks_tuned]
stks_tuned[-1] = list(stks_tuned[-1])
stks_tuned[-1][0] = 'CatBoostRegressor'
stks_tuned[-1] = tuple(stks_tuned[-1])

stks_trained_for_submissions = [(name, stk.fit(S_train,y_train)) for name, stk in tqdm(stks_tuned.copy())]

In [ ]:
stks_tuned = [Ridge(random_state=0, **max_params_rid),Lasso(random_state=0, **max_params_las),ElasticNet(random_state=0, **max_params_ela),
             ARDRegression(**max_params_ard),BayesianRidge(**max_params_bay),LGBMRegressor(random_state=0,**max_params_lgbm),CatBoostRegressor(random_state=0)]
stks_tuned = [(str(stk).split('(')[0], stk) for stk in stks_tuned]
stks_tuned[-1] = list(stks_tuned[-1])
stks_tuned[-1][0] = 'CatBoostRegressor'
stks_tuned[-1] = tuple(stks_tuned[-1])

In [ ]:
pred_results = []
for name, stk, stk_score in stks_trained:
    pred = list(stk.predict(S_dev))
    name = f'{name} \n({stk_score:.4f})'
    pred_results.append(pd.Series(pred, name=name))
ensemble_results = pd.concat(pred_results, axis=1)
ensemble_results = ensemble_results.applymap(lambda x: float(x))

# 모형의 예측값 간의 상관관계를 보기 위해 hitmap을 도식한다.
plt.figure(figsize = (8,6))
g = sns.heatmap(ensemble_results.corr(), annot=True, cmap='Blues')
g.set_title("Correlation between models")
plt.show()

In [ ]:
corr = (ensemble_results.corr().sum()-1)/(ensemble_results.corr().shape[0]-1)
names = corr.index
rmse = np.array(corr.index.str[-7:-1]).astype(float)
df = pd.DataFrame({'model': names, 'rmse': rmse, 'cor': corr})        

plt.figure(figsize=(8,6))
g = sns.scatterplot(x="cor", y="rmse", data=df, s=40, color='red')
for line in range(0, df.shape[0]):
     g.text(df.cor[line]+0.003, df.rmse[line]-0.003, 
            df.model[line], horizontalalignment='left', 
            size='medium', color='black', weight='semibold')
        
plt.xlim((df.cor.min()-0.01,df.cor.max()+0.01))
plt.ylim((df.rmse.min()-0.01,df.rmse.max()+0.01))
plt.xlabel('Mean Agreement')
plt.ylabel('RMSE')
plt.grid()
plt.show()

In [ ]:
selected = [#'LinearRegression',
            #'Ridge',
            'Lasso',
            #'ElasticNet',
            #'ARDRegression',
            #'BayesianRidge',
            #'RandomForestRegressor',
            #'XGBRegressor',
            'LGBMRegressor',
            #'CatBoostRegressor'
            ]
models_for_ensemble = [(name,stk) for name,stk,score in stks_trained if name in selected]

weights_stk = []
rmse_best = 1000
for i in tqdm(range(0, 21, 1)):
    for j in range(0, 21, 1):
        if (i+j) != 20:
            continue
        pred = (models_for_ensemble[0][1].predict(S_dev) * i + models_for_ensemble[1][1].predict(S_dev) * j)/20
        rmse = np.sqrt(mean_squared_error(y_dev, pred))
        if rmse < rmse_best:
            weights_stk = [i,j]
            rmse_best = rmse 
            print(rmse, i,j)            

print(rmse_best, weights_stk)

In [ ]:
w0,w1 = weights_stk
selected = [#'LinearRegression',
            #'Ridge',
            'Lasso',
            #'ElasticNet',
            #'ARDRegression',
            #'BayesianRidge',
            #'RandomForestRegressor',
            #'XGBRegressor',
            'LGBMRegressor',
            #'CatBoostRegressor'
            ]
models_for_ensemble = [(name,stk) for name,stk,score in stks_trained if name in selected]
stk_ensemble_val = (models_for_ensemble[0][1].predict(S_dev) * w0 + models_for_ensemble[1][1].predict(S_dev) * w1)/20

In [ ]:
weights_ds = []
rmse_best = 1000
for i in tqdm(range(0, 21, 1)):
    for j in range(0, 21, 1):
        if (i+j) != 20:
            continue
        pred = (stk_ensemble_val*i+dnn.predict(x_dev_cat).flatten()*j)/20
        rmse = np.sqrt(mean_squared_error(y_dev, pred))
        if rmse < rmse_best:
            weights_ds = [i,j]
            rmse_best = rmse 
            print(rmse, i,j)     

In [ ]:
sns.heatmap(pd.DataFrame(dict(zip(['dnn','avg','stk'],[dnn.predict(x_dev_cat).flatten(),avg_test_result,stk_ensemble_val]))).corr(), annot=True, cmap='Blues')
plt.show()

### Deployment

In [ ]:
w0,w1 = weights_stk
selected = [#'LinearRegression',
            #'Ridge',
            'Lasso',
            #'ElasticNet',
            #'ARDRegression',
            #'BayesianRidge',
            #'RandomForestRegressor',
            #'XGBRegressor',
            'LGBMRegressor',
            #'CatBoostRegressor'
            ]
models_for_ensemble = [(name,stk) for name,stk in stks_trained_for_submissions if name in selected]
stk_ensemble_results = (models_for_ensemble[0][1].predict(S_test) * w0 + models_for_ensemble[1][1].predict(S_test) * w1)/20

In [ ]:
w0, w1 = weights_ds
pd.DataFrame({'age': (stk_ensemble_val*w0+dnn.predict(x_dev_cat).flatten()*w1)/20}).to_csv('categorical_stken14laslgbm_rid_lgbm_cat_dnn6_dev.csv', index=False)

In [ ]:
w0, w1 = weights_ds
pd.DataFrame({'custid': test_id, 'age': (stk_ensemble_results*w0+dnn.predict(x_test_cat).flatten()*w1)/20}).to_csv('categorical_stken14laslgbm_rid_lgbm_cat_dnn6.csv', index=False)